In [1]:
import semantic_kernel as sk
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
print(os.getenv("OPENAI_API_BASE"))


https://alkopenai2.openai.azure.com/


In [2]:
# for this example I want a valid logger
import logging

# Create a logger
logger = logging.getLogger("SK")

# Set the logging level to INFO
logger.setLevel(logging.DEBUG)

# Create a console handler
console_handler = logging.StreamHandler()

# Set the level of the console handler to INFO
console_handler.setLevel(logging.DEBUG)

# Create a formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Set the formatter for the console handler
console_handler.setFormatter(formatter)

# Add the console handler to the logger
logger.addHandler(console_handler)


In [3]:
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextCompletion,
)

kernel = sk.Kernel(log=logger)
kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(
        "GPT42",
        os.getenv("OPENAI_API_BASE"),
        os.getenv("OPENAI_API_KEY")
    ),
)

In [4]:
# Now we need to import the plugin
from plugins.AudioVideoPlugin.AudioVideo import AudioVideo

# Now you can import the plugin importing skill directly from the function you declared
# in the plugin directory. The import_skill does not need the path, it only need an
# instance of the skill and the name of the skill
extractaudio_plugin = kernel.import_skill(AudioVideo(), skill_name="AudioVideoPlugin")

plugins_directory = "./plugins"

# Import the OrchestratorPlugin from the plugins directory.
publishing_plugin = kernel.import_semantic_skill_from_directory(
    plugins_directory, "PublishingPlugin"
)


2023-11-25 11:28:54,455 - SK - DEBUG - Importing skill AudioVideoPlugin
2023-11-25 11:28:54,456 - SK - DEBUG - Methods imported: 2
2023-11-25 11:28:54,457 - SK - DEBUG - Extracting blocks from template: I will give you a transcript of a video. The transcript contains phrases prefixed by the timestamp where the phrase starts. I want you to identify between three and ten main sections of the video. For each section you will create a brief title prefixed with the start timestamp of the section, obtained by the first phrase of the section. 

EXAMPLE ANSWER
00:00 - Title 1
00:33 - Title 2
01:23 - Title 3

[DATA]
{{$input}}


In [5]:
from pprint import pprint
# want to print all the keys of extractaudio_plugin that is a dictionary

pprint (extractaudio_plugin["ExtractAudio"])
pprint (extractaudio_plugin["TranscriptTimeline"])
pprint (publishing_plugin["VideoTimelineCreator"])

In [6]:
# Now we can use the skill
# Run the Sqrt function with the context.
result = await kernel.run_async(
    extractaudio_plugin["ExtractAudio"],
    extractaudio_plugin["TranscriptTimeline"],
    publishing_plugin["VideoTimelineCreator"],
    input_str="/Users/gianmariaricci/develop/montaggi/UpdatingSSH.mp4",
)

print(result)

Extracting auio file from video /Users/gianmariaricci/develop/montaggi/UpdatingSSH.mp4
Extracting transcript from audio file /Users/gianmariaricci/develop/montaggi/UpdatingSSH.wav


/Users/gianmariaricci/develop/github/SemanticKernelPlayground/skernel/lib/python3.11/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
2023-11-25 11:29:15,506 - SK - DEBUG - Rendering string template: I will give you a transcript of a video. The transcript contains phrases prefixed by the timestamp where the phrase starts. I want you to identify between three and ten main sections of the video. For each section you will create a brief title prefixed with the start timestamp of the section, obtained by the first phrase of the section. 

EXAMPLE ANSWER
00:00 - Title 1
00:33 - Title 2
01:23 - Title 3

[DATA]
{{$input}}
2023-11-25 11:29:15,506 - SK - DEBUG - Extracting blocks from template: I will give you a transcript of a video. The transcript contains phrases prefixed by the timestamp where the phrase starts. I want you to identify between three and ten main sections 

extracted 30 audio segments
00:00 - Introduction to Ubiki and SSHK Issue
00:39 - Explanation of SSHK Issue in Windows
01:06 - Importance of Updating SSH for Ubiki Support
01:34 - Understanding SSH Updates in Windows
02:01 - Installing Open SSH Package Separately in Windows
02:20 - Using Wingat to Install Latest SSH Version
03:14 - Verifying the Updated SSH Version
03:32 - Final Remarks on SSH Update and Ubiki Usage
03:51 - Conclusion and Video Wrap-up
